In [1]:
#calculating focal loss


import torch
import torch.nn.functional as F
import numpy as np

from losses.focal_loss import FocalLoss, reweight

def logit(p: torch.Tensor) -> torch.Tensor:
    return torch.log(p / (1 - p))


In [5]:
print('inputs shape', inputs.shape)
inputs = logit(
    torch.tensor(
        [
            [
                [0.95, 0.55, 0.12, 0.05],
                [0.09, 0.95, 0.36, 0.11],
                [0.06, 0.12, 0.56, 0.07],
                [0.09, 0.15, 0.25, 0.45],
            ]
        ],
        dtype=torch.float32,
    ).squeeze(0)
)
print('target shape', targets.shape)
targets = torch.tensor([1, 3, 0, 2], dtype=torch.long).squeeze(0)

#
print('inputs shape', inputs.shape)
print('target shape', targets.shape)

inputs shape torch.Size([4, 4])
target shape torch.Size([4])


In [11]:
print(inputs)
print(targets)
print(targets.ndim)

tensor([[ 2.9444,  0.2007, -1.9924, -2.9444],
        [-2.3136,  2.9444, -0.5754, -2.0907],
        [-2.7515, -1.9924,  0.2412, -2.5867],
        [-2.3136, -1.7346, -1.0986, -0.2007]])
tensor([1, 3, 0, 2])
1


In [13]:
#print shape before squeeze
print(logit(
    torch.tensor(
        [
            [
                [0.95, 0.55, 0.12, 0.05],
                [0.09, 0.95, 0.36, 0.11],
                [0.06, 0.12, 0.56, 0.07],
                [0.09, 0.15, 0.25, 0.45],
            ]
        ],
        dtype=torch.float32,
    )).shape)
print(torch.tensor([1, 3, 0, 2]).shape)
print(inputs.shape[0])

torch.Size([1, 4, 4])
torch.Size([4])
4


In [12]:
# get unique elements and their counts in a tensor
x = torch.tensor([1, 1, 2, 2, 3, 1, 1, 2])
unique_elements, counts = torch.unique(x, return_counts=True)

print("Unique elements:", unique_elements)
print("Counts:", counts)


Unique elements: tensor([1, 2, 3])
Counts: tensor([4, 3, 1])


In [33]:
#learn F.logSoftmax
N = targets.shape[0]
log_p = F.log_softmax(inputs, dim=1)        #output is [4,4]
N = targets.shape[0]      #batch size
log_pt = log_p[torch.arange(N), targets]    #torch.Size([4])
print("log_p: ", log_p.shape)
print("log_pt: ", log_pt.shape)


log_p:  torch.Size([4, 4])
log_pt:  torch.Size([4])
tensor([0.0599, 0.0062, 0.0412, 0.2336])


In [34]:
pt = log_pt.exp()
print(pt)

sample_weights = pt

tensor([0.0599, 0.0062, 0.0412, 0.2336])


In [5]:
import torch
z = torch.tensor([[1], [2]])
z * 3


tensor([[3],
        [6]])

In [30]:
import torch
import torch.nn.functional as F

W1 = torch.tensor([[1.5], [0.5]])
b1 = torch.tensor([[0.], [1.]])
W2 = torch.tensor([[1.,2.],[2.,1.]])
b2= torch.tensor([[0.], [1.]])
W3 = torch.tensor([[1.,1.]])
b3 = torch.tensor([[-1.]])
print(W1, b1, W2, b2, W3, b3)

#x = 2--------
x = torch.tensor([[2.]])

h = W3 @ F.relu(W2 @ F.relu(W1 * x  + b1) + b2) + b3
h

tensor([[1.5000],
        [0.5000]]) tensor([[0.],
        [1.]]) tensor([[1., 2.],
        [2., 1.]]) tensor([[0.],
        [1.]]) tensor([[1., 1.]]) tensor([[-1.]])


tensor([[15.]])

In [76]:
def relu_deriv(x):
    deriv =  (x > 0).float()
    return deriv

deriv = relu_deriv(b2)
print(deriv)
print(torch.diag(deriv.squeeze()))      #need 1D tensor input into diag

tensor([[0.],
        [1.]])
tensor([[0., 0.],
        [0., 1.]])


In [91]:
def relu_deriv(x):
    deriv =  (x > 0).float()
    #return as diag tensor
    return torch.diag(deriv.squeeze())

deriv = relu_deriv(b2)
# print(deriv)

#x value------
x0 = 1.
x = torch.tensor([[x0]])
print("x: ", x.item())

W1 = torch.tensor([[1.5], [0.5]])
b1 = torch.tensor([[0.], [1.]])
W2 = torch.tensor([[1.,2.],[2.,1.]])
b2= torch.tensor([[0.], [1.]])
W3 = torch.tensor([[1.,1.]])
b3 = torch.tensor([[-1.]])

z1 = W1 * x + b1
a1 = torch.relu(z1)
z2 = W2 @ a1 + b2
a2 = torch.relu(z2)
z3 = W3 @ a2 + b3

print('H first', h)

def calc_h(x):
    z1 = W1 * x + b1
    a1 = torch.relu(z1)
    z2 = W2 @ a1 + b2
    a2 = torch.relu(z2)
    h = W3 @ a2 + b3
    return h.item()

h = calc_h(x)
print("h:", h)

dz1_dx = W1
da1_dz1 = relu_deriv(z1)
dz2_da1 = W2
da2_dz2 = relu_deriv(z2)
dz3_da2 = W3

dh_dx = dz3_da2 @ da2_dz2 @ dz2_da1 @ da1_dz1 @ dz1_dx
dh_dx = dh_dx.item()
print("dh:", dh_dx)

#calc W, b
W = dh_dx
b = h- W * x0
print("W: ", W)
print("b: ", b)


x:  1.0
H first 1.5
h: 9.0
dh: 6.0
W:  6.0
b:  3.0


In [75]:
#calc W, b
W = dh_dx
b = h - W * x